<a href="https://colab.research.google.com/github/ErrorOfTheGeneration/pyBotTG_google/blob/main/pyBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random   # для рандомных ответов из словаря ответов
import nltk     # для работы с опечатками
import json


In [22]:
# далее обозначено зачем 1-я библиотека. 2-ю погуглить
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
with open('/content/BIG_BOT_CONFIG.json', 'r') as f:  # json для хранения словарей (поискать справку)
  BOT_CONFIG = json.load(f)
# словарь словарей. Нужен нам для того, чтобы определять, что хочет пользователь
# и соттветствующе отвечать ему
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Хай', 'Добрый день', 'здравствуй', 'здравствуйте',
#                          'приветик', 'здрасте', 'здорова'], # примеры приветствия
#             'resposes': ['Доброго времени суток', 'Привет', 'приветствую', 'Здравствуй']  #ответы бота
#         },
#         'bye': {
#             'examples': ['Пока', 'До свидания', 'Увидимся', 'Счастливо', 'до новых встреч'],
#             'resposes': ['Счастливо', 'Хаюхай', 'Удачного дня', 'Приходите ещё', 'Будьте здоровы', 'До новых встреч!']
#         }
#     }
# }

In [ ]:
len(BOT_CONFIG['intents'])

In [4]:


def clean(text): #принимает текст, возвращает чищенный текст ниж.регистр и без знаков препинания
  return ''.join([simbol for simbol in text.lower() if simbol in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '])

# функция match нужна для того, чтобы бот не был чувствителен к регистру или знакам препинания

# nltk позволяет вычислять расстояние Ливенштейна 
# между строками, т.е. кол-во редакторских исправлений, 
# чтобы из одного слова получить другое
def match(example, text):      
  return nltk.edit_distance(clean(text),clean(example)) / len(example) < 0.4

# пробегаем по всему нашему словарю и ищем, в какой интент (намерение) попадает
# введённое пользователем слово
def get_intent(text):
  for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
      if match(example, text):
        return random.choice(value['resposes'])
  return'Я не понимаю, чего от меня хотят'

In [ ]:
question = input()            # просим пользователя ввести
answer = get_intent(question)
print(answer)

In [ ]:
while question != 'выход':
  question = input()            
  answer = get_intent(question)
  print(answer)

возьмем библиотеку scikit-learn. В ней есть векторайзеры - специальные объекты,
которые превращают тексты в вектора

In [ ]:
# Сделаем дата-сет (набор данных для машинного обучения) из файла BOT_CONFIG
# нам нужны пары объект-ответ
# икс - для объектов обучающей выборки. там - вектора всех текстов 
# игрек - ответы для всех векторов, записанные в том же самом порядке
# X = [v1, v2, v3]
# Y = [int1, int2, int3]

In [14]:
X = []
y = []

for intent, value in BOT_CONFIG['intents'].items():
  examples = set(value['examples']) # *1
  X = X + value['examples']
  y = y + [intent] * len(value['examples'])
# *1 преобразуем список в множество на случай неуникальных экземплс, чтобы не 
# не затруднять обучение машине. Теперь элементы не повторяются

In [27]:
# Векторизуем дата-сет. Эти вектора можно подавать на вход модели
vectorizer = TfidfVectorizer() # ngram_range - с помощью этого пар-ра работаем со словосочет.
X_transformed = vectorizer.fit_transform(X)

In [ ]:
# теперь мы можем делать предсказания (predict)
# text = 'пока'
# classifier.predict(vectorizer.transform([text]))

Заметим что векторайзер не зависит от регистра. Любому тексту он может поставить в соответствие вектор. И всем текстам надо поставить вектора, чтобы их показать модели машинного обучения, а она на основе векторов попытается понять, как зависит класс текста, намерение пользователя, от того, на каких местах какие числа в этих векторах располагаются.


Нам нужно замерить качетво классификации. Ибо вдруг на этих 2х примерах правильно работает, а в остальных случаях нет.
Кач-во классификации замеряют так:берут большую выборку (наше X_transformed), делят ее вкакой-то пропорции (например 80% на 20%, или 66% и 34%) и на  2/3 обучают модель, а тестируют на 1/3. Это делают, чтобы понять, поняла ли модель общий закон "...". Когда модель запоминает некоторые конкретные признаки, которые не обобщаются,и которые будут бесполезны за пределами этого дата-сета,называется переобучением. Чтобы его избежать, исп. разделение выборки. Для удобного разделения выборки тоже есть функция в сайкет лёрне sklearn.model_selection.train_test_split

В train_test_split надо передать саму выборку X = X_transformed, 

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

In [29]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train) # обучаем

RandomForestClassifier()

In [30]:
# можем посмотреть, какого результата мы добились на тестовой выборке(отложенной)
# т.е. которую мы отложили чтобы замерить качество. Чем больше обучающих примеров, 
# тем выше будет качество
classifier.score(X_test, y_test)

0.5

In [31]:
classifier.score(X_train, y_train)

1.0